# Imports e inicialización del servidor

In [1]:
import mysql.connector
import os, getpass
import pandas as pd
import numpy as np
import cvxpy as cp
from mysql.connector import Error
from itertools import combinations
from ast import literal_eval
from collections import defaultdict
from scipy.spatial import distance

In [3]:
# Definición de parámetros de conexión
connection_params = {
    'host': 'localhost',
    'user': 'cmescobar',
    'database': 'foodb',
    'password': getpass.getpass(prompt='Introduzca la contraseña: ')
}

try:
    connection = mysql.connector.connect(**connection_params)

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()

except Error as e:
    print("Error while connecting to MySQL", e)

Introduzca la contraseña: ········
Connected to MySQL Server version  8.0.26


# Diccionarios de mapeo de las entradas de la tabla objetivo a cada tabla de alimentos

In [4]:
nutrients_public_id = list()
compounds_public_id = list()

# Definición de las componentes de interés en una bebida energética
with open('interest_drink_comps_codes.csv', 'r', encoding='utf8') as file:
    for line in file:
        data = line.strip().split(';')
        
        # Revisando el código de interés
        if 'FDBN' in data[2]:
            nutrients_public_id.append((data[0], data[2]))
        else:
            compounds_public_id.append((data[0], data[2]))
            
# Finalmente, se eliminan los duplicados
compounds_public_id = list(set(compounds_public_id))
compounds_public_id

# Diccionario que permite mapear el public_id al nombre de tabla
with open('Summary/publicId_to_objTable.txt', 'w', encoding='utf8') as file:
    publicId_to_objTable = {i[1]: i[0] for i in compounds_public_id}
    file.write(str(publicId_to_objTable))

In [5]:
compounds_public_id

[('Selenium, Se', 'FDB013400'),
 ('Potassium, K', 'FDB003521'),
 ('Magnesium, Mg', 'FDB003518'),
 ('Vitamin B12, added', 'FDB022886'),
 ('Riboflavin', 'FDB012160'),
 ('Zinc, Zn', 'FDB003729'),
 ('Vitamin C', 'FDB001223'),
 ('Vitamin B6', 'FDB000574'),
 ('Calcium, Ca', 'FDB003513'),
 ('Vitamin K', 'FDB023087'),
 ('Phosphorus, P', 'FDB003520'),
 ('Copper, Cu', 'FDB003582'),
 ('Iron, Fe', 'FDB016251'),
 ('Caffeine', 'FDB002100'),
 ('Niacin', 'FDB001014'),
 ('Thiamin', 'FDB008424'),
 ('Water', 'FDB013390'),
 ('Folic acid', 'FDB014504'),
 ('Choline', 'FDB000710'),
 ('Sodium, Na', 'FDB003523'),
 ('Sugars', 'FDB003715')]

## Consulta para linkear nombre de la base con los de la tabla objetivo

In [6]:
query = f'''SELECT id, name, public_id
    FROM compounds
    WHERE public_id IN {list(zip(*compounds_public_id))[1]}
    ;
'''

# Ejecutando la query
cursor.execute(query)

# Desplegando la información: Obteniendo la lista de foods_id
compounds_id_name = [i for i in cursor.fetchall()]

# Diccionario que permite mapear el source_id al public_id
with open('Summary/sourceId_to_publicId.txt', 'w', encoding='utf8') as file:
    sourceId_to_publicId = {i[0]: i[2] for i in compounds_id_name}
    file.write(str(sourceId_to_publicId))

# Lectura de los archivos de datos

In [14]:
# Definición de un diccionario de tablas
tables_dict = dict()

# Obteniendo los archivos nutricionales de la carpeta food_compound_mini
files = [i for i in os.listdir('Summary/Food_Compound_Mini') 
         if i.endswith('.csv')]

# Para cada archivo en la carpeta de tablas nutricionales
for file in files:
    # Se obtiene el dataframe de pandas
    df = pd.read_csv(f'Summary/Food_Compound_Mini/{file}', sep=';', 
                     header=0, encoding='utf8', decimal='.')
    # Dataframe acortado
    df_short = df[['source_id', 'orig_content']].replace({'None': 0.0})
    
    for i in range(len(df_short.iloc[:, 0])):
        # Definición del source_id_i
        src_i = df_short.iloc[i, 0]
        
        # Mapeando al diccionario de interés
        df_short.iloc[i, 0] = publicId_to_objTable[sourceId_to_publicId[src_i]]
        
        # Mapeando también los valores a normalizados
        df_short.iloc[i, 1] = float(df_short.iloc[i, 1]) / 1000 / 100
    
    # Ordenando el dataframe
    df_short = df_short.sort_values(by='source_id', axis='index', ignore_index=True)
    tables_dict[file[:-4]] = df_short
    
print(file)
df_short

Turmeric.csv


,source_id,orig_content
0,Caffeine,0.0
1,"Calcium, Ca",0.00183
2,Choline,0.000492
3,"Copper, Cu",0.000006
4,Folic acid,0.0
5,"Iron, Fe",0.000414
6,"Magnesium, Mg",0.00193
7,Niacin,0.000051
8,"Phosphorus, P",0.00268
9,"Potassium, K",0.02525


In [8]:
# Definición de la tabla objetivo
obj_table = pd.read_csv('Summary/objective_table.csv', sep=';', 
                     header=0, encoding='utf8', decimal='.')
obj_table = obj_table.sort_values(by='Compound', axis='index', ignore_index=True)
obj_table = obj_table.drop(labels=10, axis=0)
obj_table

,Compound,proportion
0,Caffeine,2.950000e-04
1,"Calcium, Ca",5.357143e-05
2,Choline,1.497696e-06
3,"Copper, Cu",2.966590e-08
4,Folic acid,2.385714e-07
5,"Iron, Fe",2.160138e-07
6,"Magnesium, Mg",3.928571e-05
7,Niacin,4.606020e-05
8,"Phosphorus, P",1.285714e-05
9,"Potassium, K",3.785714e-05


In [9]:
np.array(obj_table.iloc[:, 1])

array([2.95000000e-04, 5.35714286e-05, 1.49769585e-06, 2.96658986e-08,
       2.38571429e-07, 2.16013825e-07, 3.92857143e-05, 4.60601959e-05,
       1.28571429e-05, 3.78571429e-05, 3.18519585e-06, 1.14343318e-08,
       4.45000000e-04, 9.55069124e-02, 1.75979263e-07, 1.69182028e-08,
       9.97839862e-06, 7.14285714e-05, 5.76036866e-11, 8.95529954e-01,
       9.50460829e-07])

# Implementación modelo

In [10]:
# Número de ingredientes máximos por receta
compounds_max = 5

# Definición de los ingredientes
ingredients = list(tables_dict.keys())

# Definición de las diferentes combinaciones de recetas en base a los ingredientes
recipes = combinations(ingredients, compounds_max)

In [11]:
def get_recipe_portions(ingredients, obj_table_pandas, food_tables_dict,
                        solver=cp.ECOS, verbose=False):
    def _objective_table():
        # Definición de la tabla nutricional objetivo
        return np.array(obj_table_pandas.iloc[:, 1], dtype=float)
    
    
    def _recipes_table():
        # Creación del problema de optimización para la definición de la receta
        recipe_matrix = list()

        for ing_i in ingredients:
            # Obteniendo el valor de las recetas
            ing_values_i = np.array(food_tables_dict[ing_i].iloc[:, 1], dtype=float)
            
            # Agregando a la matriz de recetas
            recipe_matrix.append(ing_values_i)
    
        return recipe_matrix
    
    
    def _model():
        # Variable de cantidad
        x = cp.Variable(len(ingredients))
        
        # Parámetros
        T_obj = cp.Parameter(shape=obj_table.shape)
        T_obj.value = obj_table
        T_rec = cp.Parameter(shape=recipe_matrix_arr.shape)
        T_rec.value = recipe_matrix_arr

        # Restricciones
        constraints = [x >= 0]

        # Definición de la función objetivo
        objective_func = cp.Minimize(cp.sum_squares((T_obj - T_rec @ x)))

        # Definición del problema de optimización
        prob = cp.Problem(objective=objective_func, constraints=constraints)
        
        # Calculando el óptimo
        result = prob.solve(solver=solver, verbose=verbose)
        
        return result, x.value
     
    
    def _cosine_similarity(a, b):
        return 1 - distance.cosine(a, b)
    
    
    def _euclidean_distance(a, b):
        return np.sqrt(np.sum((a - b) ** 2))
    
    
    # Obteniendo las tablas
    obj_table = _objective_table()
    recipe_matrix = _recipes_table()
    
    # Transformando a arrays
    recipe_matrix_arr = np.array(recipe_matrix).T
    
    # Obtención de los valores del modelo
    result, x = _model()
    
    # Obteniendo la distancia euclidiana y coseno
    euclidean_dist = _euclidean_distance(obj_table, recipe_matrix_arr @ x)
    cosine_sim     = _cosine_similarity(obj_table, recipe_matrix_arr @ x)
    
    return result, x, (euclidean_dist, cosine_sim)

In [12]:
# Definición de la información de las recetas
recipes_results = list()

# Para cada receta en recetas
for recipe_i in recipes:
    print('Getting recipe: ', recipe_i)
    # Obtención de las porciones de las recetas
    result, x, metrics = get_recipe_portions(recipe_i, 
                                             obj_table, 
                                             tables_dict,
                                             solver=cp.CVXOPT, 
                                             verbose=False)
    
    
    recipes_results.append((recipe_i, result, x, metrics))

Getting recipe:  ('Chocolate', 'Cinnamon', 'Coffee', 'Common sage', 'Common thyme')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Coffee', 'Common sage', 'Common wheat')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Coffee', 'Common sage', 'Ginger')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Coffee', 'Common sage', 'Ginkgo nuts')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Coffee', 'Common sage', 'Green Tea')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Coffee', 'Common sage', 'Hibiscus tea')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Coffee', 'Common sage', 'Horseradish tree')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Coffee', 'Common sage', 'Maitake')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Coffee', 'Common sage', 'Pepper')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Coffee', 'Common sage', 'Rosemary')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Coffee', 'Common sage', 'Salt')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Coffee', 'Common sage', 'Shiitake')
Getting recipe:  ('Chocol

Getting recipe:  ('Chocolate', 'Cinnamon', 'Common sage', 'Common wheat', 'Green Tea')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Common sage', 'Common wheat', 'Hibiscus tea')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Common sage', 'Common wheat', 'Horseradish tree')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Common sage', 'Common wheat', 'Maitake')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Common sage', 'Common wheat', 'Pepper')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Common sage', 'Common wheat', 'Rosemary')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Common sage', 'Common wheat', 'Salt')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Common sage', 'Common wheat', 'Shiitake')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Common sage', 'Common wheat', 'Turmeric')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Common sage', 'Ginger', 'Ginkgo nuts')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Common sage', 'Ginger', 'Green Tea')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Common sage'

Getting recipe:  ('Chocolate', 'Cinnamon', 'Common thyme', 'Hibiscus tea', 'Salt')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Common thyme', 'Hibiscus tea', 'Shiitake')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Common thyme', 'Hibiscus tea', 'Turmeric')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Common thyme', 'Horseradish tree', 'Maitake')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Common thyme', 'Horseradish tree', 'Pepper')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Common thyme', 'Horseradish tree', 'Rosemary')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Common thyme', 'Horseradish tree', 'Salt')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Common thyme', 'Horseradish tree', 'Shiitake')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Common thyme', 'Horseradish tree', 'Turmeric')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Common thyme', 'Maitake', 'Pepper')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Common thyme', 'Maitake', 'Rosemary')
Getting recipe:  ('Chocolate', 'Cinnamon',

Getting recipe:  ('Chocolate', 'Cinnamon', 'Ginger', 'Hibiscus tea', 'Rosemary')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Ginger', 'Hibiscus tea', 'Salt')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Ginger', 'Hibiscus tea', 'Shiitake')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Ginger', 'Hibiscus tea', 'Turmeric')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Ginger', 'Horseradish tree', 'Maitake')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Ginger', 'Horseradish tree', 'Pepper')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Ginger', 'Horseradish tree', 'Rosemary')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Ginger', 'Horseradish tree', 'Salt')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Ginger', 'Horseradish tree', 'Shiitake')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Ginger', 'Horseradish tree', 'Turmeric')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Ginger', 'Maitake', 'Pepper')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Ginger', 'Maitake', 'Rosemary')
Getting recipe:  ('Chocolat

Getting recipe:  ('Chocolate', 'Cinnamon', 'Hibiscus tea', 'Salt', 'Shiitake')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Hibiscus tea', 'Salt', 'Turmeric')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Hibiscus tea', 'Shiitake', 'Turmeric')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Horseradish tree', 'Maitake', 'Pepper')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Horseradish tree', 'Maitake', 'Rosemary')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Horseradish tree', 'Maitake', 'Salt')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Horseradish tree', 'Maitake', 'Shiitake')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Horseradish tree', 'Maitake', 'Turmeric')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Horseradish tree', 'Pepper', 'Rosemary')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Horseradish tree', 'Pepper', 'Salt')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Horseradish tree', 'Pepper', 'Shiitake')
Getting recipe:  ('Chocolate', 'Cinnamon', 'Horseradish tree', 'Pepper', 'Turmeric')

...

Getting recipe:  ('Ginger', 'Ginkgo nuts', 'Hibiscus tea', 'Horseradish tree', 'Rosemary')
Getting recipe:  ('Ginger', 'Ginkgo nuts', 'Hibiscus tea', 'Horseradish tree', 'Salt')
Getting recipe:  ('Ginger', 'Ginkgo nuts', 'Hibiscus tea', 'Horseradish tree', 'Shiitake')
Getting recipe:  ('Ginger', 'Ginkgo nuts', 'Hibiscus tea', 'Horseradish tree', 'Turmeric')
Getting recipe:  ('Ginger', 'Ginkgo nuts', 'Hibiscus tea', 'Maitake', 'Pepper')
Getting recipe:  ('Ginger', 'Ginkgo nuts', 'Hibiscus tea', 'Maitake', 'Rosemary')
Getting recipe:  ('Ginger', 'Ginkgo nuts', 'Hibiscus tea', 'Maitake', 'Salt')
Getting recipe:  ('Ginger', 'Ginkgo nuts', 'Hibiscus tea', 'Maitake', 'Shiitake')
Getting recipe:  ('Ginger', 'Ginkgo nuts', 'Hibiscus tea', 'Maitake', 'Turmeric')
Getting recipe:  ('Ginger', 'Ginkgo nuts', 'Hibiscus tea', 'Pepper', 'Rosemary')
Getting recipe:  ('Ginger', 'Ginkgo nuts', 'Hibiscus tea', 'Pepper', 'Salt')
Getting recipe:  ('Ginger', 'Ginkgo nuts', 'Hibiscus tea', 'Pepper', 'Shiitake

Getting recipe:  ('Ginger', 'Green Tea', 'Pepper', 'Shiitake', 'Turmeric')
Getting recipe:  ('Ginger', 'Green Tea', 'Rosemary', 'Salt', 'Shiitake')
Getting recipe:  ('Ginger', 'Green Tea', 'Rosemary', 'Salt', 'Turmeric')
Getting recipe:  ('Ginger', 'Green Tea', 'Rosemary', 'Shiitake', 'Turmeric')
Getting recipe:  ('Ginger', 'Green Tea', 'Salt', 'Shiitake', 'Turmeric')
Getting recipe:  ('Ginger', 'Hibiscus tea', 'Horseradish tree', 'Maitake', 'Pepper')
Getting recipe:  ('Ginger', 'Hibiscus tea', 'Horseradish tree', 'Maitake', 'Rosemary')
Getting recipe:  ('Ginger', 'Hibiscus tea', 'Horseradish tree', 'Maitake', 'Salt')
Getting recipe:  ('Ginger', 'Hibiscus tea', 'Horseradish tree', 'Maitake', 'Shiitake')
Getting recipe:  ('Ginger', 'Hibiscus tea', 'Horseradish tree', 'Maitake', 'Turmeric')
Getting recipe:  ('Ginger', 'Hibiscus tea', 'Horseradish tree', 'Pepper', 'Rosemary')
Getting recipe:  ('Ginger', 'Hibiscus tea', 'Horseradish tree', 'Pepper', 'Salt')
Getting recipe:  ('Ginger', 'Hib

Getting recipe:  ('Ginkgo nuts', 'Green Tea', 'Horseradish tree', 'Rosemary', 'Shiitake')
Getting recipe:  ('Ginkgo nuts', 'Green Tea', 'Horseradish tree', 'Rosemary', 'Turmeric')
Getting recipe:  ('Ginkgo nuts', 'Green Tea', 'Horseradish tree', 'Salt', 'Shiitake')
Getting recipe:  ('Ginkgo nuts', 'Green Tea', 'Horseradish tree', 'Salt', 'Turmeric')
Getting recipe:  ('Ginkgo nuts', 'Green Tea', 'Horseradish tree', 'Shiitake', 'Turmeric')
Getting recipe:  ('Ginkgo nuts', 'Green Tea', 'Maitake', 'Pepper', 'Rosemary')
Getting recipe:  ('Ginkgo nuts', 'Green Tea', 'Maitake', 'Pepper', 'Salt')
Getting recipe:  ('Ginkgo nuts', 'Green Tea', 'Maitake', 'Pepper', 'Shiitake')
Getting recipe:  ('Ginkgo nuts', 'Green Tea', 'Maitake', 'Pepper', 'Turmeric')
Getting recipe:  ('Ginkgo nuts', 'Green Tea', 'Maitake', 'Rosemary', 'Salt')
Getting recipe:  ('Ginkgo nuts', 'Green Tea', 'Maitake', 'Rosemary', 'Shiitake')
Getting recipe:  ('Ginkgo nuts', 'Green Tea', 'Maitake', 'Rosemary', 'Turmeric')
Getting

Getting recipe:  ('Green Tea', 'Hibiscus tea', 'Horseradish tree', 'Rosemary', 'Turmeric')
Getting recipe:  ('Green Tea', 'Hibiscus tea', 'Horseradish tree', 'Salt', 'Shiitake')
Getting recipe:  ('Green Tea', 'Hibiscus tea', 'Horseradish tree', 'Salt', 'Turmeric')
Getting recipe:  ('Green Tea', 'Hibiscus tea', 'Horseradish tree', 'Shiitake', 'Turmeric')
Getting recipe:  ('Green Tea', 'Hibiscus tea', 'Maitake', 'Pepper', 'Rosemary')
Getting recipe:  ('Green Tea', 'Hibiscus tea', 'Maitake', 'Pepper', 'Salt')
Getting recipe:  ('Green Tea', 'Hibiscus tea', 'Maitake', 'Pepper', 'Shiitake')
Getting recipe:  ('Green Tea', 'Hibiscus tea', 'Maitake', 'Pepper', 'Turmeric')
Getting recipe:  ('Green Tea', 'Hibiscus tea', 'Maitake', 'Rosemary', 'Salt')
Getting recipe:  ('Green Tea', 'Hibiscus tea', 'Maitake', 'Rosemary', 'Shiitake')
Getting recipe:  ('Green Tea', 'Hibiscus tea', 'Maitake', 'Rosemary', 'Turmeric')
Getting recipe:  ('Green Tea', 'Hibiscus tea', 'Maitake', 'Salt', 'Shiitake')
Getting 

Getting recipe:  ('Horseradish tree', 'Maitake', 'Rosemary', 'Salt', 'Turmeric')
Getting recipe:  ('Horseradish tree', 'Maitake', 'Rosemary', 'Shiitake', 'Turmeric')
Getting recipe:  ('Horseradish tree', 'Maitake', 'Salt', 'Shiitake', 'Turmeric')
Getting recipe:  ('Horseradish tree', 'Pepper', 'Rosemary', 'Salt', 'Shiitake')
Getting recipe:  ('Horseradish tree', 'Pepper', 'Rosemary', 'Salt', 'Turmeric')
Getting recipe:  ('Horseradish tree', 'Pepper', 'Rosemary', 'Shiitake', 'Turmeric')
Getting recipe:  ('Horseradish tree', 'Pepper', 'Salt', 'Shiitake', 'Turmeric')
Getting recipe:  ('Horseradish tree', 'Rosemary', 'Salt', 'Shiitake', 'Turmeric')
Getting recipe:  ('Maitake', 'Pepper', 'Rosemary', 'Salt', 'Shiitake')
Getting recipe:  ('Maitake', 'Pepper', 'Rosemary', 'Salt', 'Turmeric')
Getting recipe:  ('Maitake', 'Pepper', 'Rosemary', 'Shiitake', 'Turmeric')
Getting recipe:  ('Maitake', 'Pepper', 'Salt', 'Shiitake', 'Turmeric')
Getting recipe:  ('Maitake', 'Rosemary', 'Salt', 'Shiitake'

In [13]:
recipes_results

[(('Chocolate', 'Cinnamon', 'Coffee', 'Common sage', 'Common thyme'),
  0.008374899956282066,
  array([1.39009803e-07, 8.26526076e+00, 1.62444574e-01, 1.43826112e-07,
         1.32494278e-07]),
  (0.09151447948976198, 0.9948238968187602)),
 (('Chocolate', 'Cinnamon', 'Coffee', 'Common sage', 'Common wheat'),
  0.0030647248255116786,
  array([3.80971829e-04, 7.49503275e-03, 1.60814619e-01, 1.18399789e-05,
         8.04988580e+00]),
  (0.05535995687779822, 0.9981089627115692)),
 (('Chocolate', 'Cinnamon', 'Coffee', 'Common sage', 'Ginger'),
  0.007670219570651025,
  array([7.52890256e-06, 6.26916079e+00, 3.44859301e-02, 1.45173797e-06,
         2.21642211e+00]),
  (0.08757978973856369, 0.9952604614377604)),
 (('Chocolate', 'Cinnamon', 'Coffee', 'Common sage', 'Ginkgo nuts'),
  0.0047579430300814275,
  array([2.07624706e-05, 2.74669823e+00, 1.61130383e-01, 5.94258386e-06,
         4.75179068e+00]),
  (0.06897784448706286, 0.9970626529740453)),
 (('Chocolate', 'Cinnamon', 'Coffee', 'Common

# Ordenando por similaridad de recetas

## Distancia euclidiana

In [13]:
recipes_ordered = [i for i in recipes_results]
recipes_ordered.sort(key=lambda x: x[3][0])

In [14]:
for num, i in enumerate(recipes_ordered):
    # Formateando el texto
    text = ','.join([t for t in i[0]])
    
    # Desplegando los resultados
    print(f'{num+1};{text};{i[3][0]:.10f}')

1;Chocolate,Hibiscus tea,Horseradish tree,Maitake,Salt;0.0005037898
2;Chocolate,Green Tea,Hibiscus tea,Horseradish tree,Salt;0.0005042088
3;Chocolate,Common wheat,Hibiscus tea,Rosemary,Salt;0.0005043066
4;Chocolate,Common wheat,Hibiscus tea,Horseradish tree,Salt;0.0005045710
5;Chocolate,Hibiscus tea,Pepper,Salt,Shiitake;0.0005053224
6;Chocolate,Green Tea,Hibiscus tea,Pepper,Salt;0.0005055516
7;Chocolate,Ginger,Hibiscus tea,Pepper,Salt;0.0005065919
8;Chocolate,Common sage,Hibiscus tea,Pepper,Salt;0.0005072421
9;Chocolate,Ginger,Ginkgo nuts,Hibiscus tea,Salt;0.0005084523
10;Chocolate,Ginkgo nuts,Hibiscus tea,Salt,Shiitake;0.0005088213
11;Chocolate,Ginger,Hibiscus tea,Maitake,Salt;0.0005102475
12;Chocolate,Ginger,Hibiscus tea,Horseradish tree,Salt;0.0005105896
13;Chocolate,Green Tea,Hibiscus tea,Salt,Turmeric;0.0005107502
14;Chocolate,Hibiscus tea,Maitake,Pepper,Salt;0.0005117100
15;Chocolate,Common sage,Hibiscus tea,Salt,Shiitake;0.0005119139
16;Chocolate,Coffee,Hibiscus tea,Pepper,Salt;

3904;Common wheat,Maitake,Rosemary,Salt,Turmeric;0.0503495839
3905;Cinnamon,Maitake,Rosemary,Salt,Turmeric;0.0503496074
3906;Ginkgo nuts,Maitake,Rosemary,Salt,Turmeric;0.0503496219
3907;Coffee,Common wheat,Green Tea,Shiitake,Turmeric;0.0553596468
3908;Chocolate,Coffee,Common wheat,Pepper,Turmeric;0.0553596505
3909;Coffee,Common thyme,Common wheat,Pepper,Shiitake;0.0553596513
3910;Chocolate,Coffee,Common wheat,Rosemary,Salt;0.0553596516
3911;Chocolate,Coffee,Common wheat,Green Tea,Pepper;0.0553596519
3912;Coffee,Common sage,Common wheat,Pepper,Shiitake;0.0553596528
3913;Coffee,Common sage,Common thyme,Common wheat,Pepper;0.0553596529
3914;Coffee,Common wheat,Ginger,Ginkgo nuts,Pepper;0.0553596532
3915;Chocolate,Coffee,Common wheat,Ginger,Pepper;0.0553596532
3916;Coffee,Common wheat,Ginkgo nuts,Pepper,Shiitake;0.0553596544
3917;Chocolate,Coffee,Common sage,Common wheat,Turmeric;0.0553596545
3918;Coffee,Common sage,Common wheat,Ginger,Turmeric;0.0553596548
3919;Coffee,Common sage,Common w

## Distancia coseno

In [15]:
recipes_ordered = [i for i in recipes_results]
recipes_ordered.sort(key=lambda x: x[3][1], reverse=True)

In [16]:
for num, i in enumerate(recipes_ordered):
    # Formateando el texto
    text = ','.join([t for t in i[0]])
    
    # Desplegando los resultados
    print(f'{num+1};{text};{i[3][1]:.10f}')

1;Chocolate,Hibiscus tea,Horseradish tree,Maitake,Salt;0.9999998435
2;Chocolate,Green Tea,Hibiscus tea,Horseradish tree,Salt;0.9999998433
3;Chocolate,Common wheat,Hibiscus tea,Rosemary,Salt;0.9999998432
4;Chocolate,Common wheat,Hibiscus tea,Horseradish tree,Salt;0.9999998431
5;Chocolate,Hibiscus tea,Pepper,Salt,Shiitake;0.9999998426
6;Chocolate,Green Tea,Hibiscus tea,Pepper,Salt;0.9999998424
7;Chocolate,Ginger,Hibiscus tea,Pepper,Salt;0.9999998420
8;Chocolate,Common sage,Hibiscus tea,Pepper,Salt;0.9999998414
9;Chocolate,Ginger,Ginkgo nuts,Hibiscus tea,Salt;0.9999998406
10;Chocolate,Ginkgo nuts,Hibiscus tea,Salt,Shiitake;0.9999998404
11;Chocolate,Ginger,Hibiscus tea,Maitake,Salt;0.9999998395
12;Chocolate,Ginger,Hibiscus tea,Horseradish tree,Salt;0.9999998393
13;Chocolate,Green Tea,Hibiscus tea,Salt,Turmeric;0.9999998392
14;Chocolate,Hibiscus tea,Maitake,Pepper,Salt;0.9999998386
15;Chocolate,Common sage,Hibiscus tea,Salt,Shiitake;0.9999998385
16;Chocolate,Coffee,Hibiscus tea,Pepper,Salt;

3648;Cinnamon,Common thyme,Common wheat,Maitake,Shiitake;0.9986169073
3649;Common thyme,Common wheat,Maitake,Rosemary,Shiitake;0.9986169073
3650;Common thyme,Ginkgo nuts,Maitake,Rosemary,Shiitake;0.9986169073
3651;Cinnamon,Common thyme,Ginkgo nuts,Maitake,Shiitake;0.9986169072
3652;Common thyme,Common wheat,Ginkgo nuts,Maitake,Shiitake;0.9986169070
3653;Common thyme,Ginkgo nuts,Horseradish tree,Maitake,Shiitake;0.9986169070
3654;Common thyme,Green Tea,Maitake,Rosemary,Shiitake;0.9986169069
3655;Common sage,Common thyme,Maitake,Rosemary,Shiitake;0.9986169068
3656;Common sage,Common thyme,Horseradish tree,Maitake,Shiitake;0.9986169068
3657;Common sage,Common thyme,Common wheat,Maitake,Shiitake;0.9986169065
3658;Common thyme,Horseradish tree,Maitake,Rosemary,Shiitake;0.9986169059
3659;Common thyme,Ginkgo nuts,Green Tea,Maitake,Shiitake;0.9986169059
3660;Common sage,Common thyme,Green Tea,Maitake,Shiitake;0.9986169059
3661;Common thyme,Maitake,Rosemary,Shiitake,Turmeric;0.9986169054
3662;C